# Competitive Intelligence Dashboard

Automated competitor analysis using LLM workflows.

**Day 5 Concepts:** Multi-step LLM workflows, structured JSON outputs, content aggregation

## Imports

In [3]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

# Load environment variables
load_dotenv(override=True)

MODEL = 'gpt-5-nano'
openai = OpenAI()

## Competitive Intelligence Class

In [4]:
class CompetitiveIntelligence:
    """
    Analyzes competitor websites and generates intelligence reports
    Day 5 concepts: Multi-step LLM workflows, structured outputs, content aggregation
    """
    
    def __init__(self, model: str = "gpt-5-nano"):
        self.model = model
        
        # System prompt for link selection (Day 5 pattern)
        self.link_system_prompt = """
        You are provided with a list of links from a competitor website.
        Select links relevant for competitive analysis:
        - Product pages
        - Pricing pages
        - About/Company pages
        - Feature pages
        - Case studies
        Do not include: Terms of Service, Privacy, email links.
        Respond in JSON format:
        {
            "links": [
                {"type": "product page", "url": "https://..."},
                {"type": "pricing page", "url": "https://..."}
            ]
        }
        """
        
        # System prompt for data extraction (Day 5 pattern)
        self.extraction_system_prompt = """
        You are analyzing a competitor company website.
        Extract key competitive intelligence:
        - Company name
        - Products/services
        - Key features
        - Pricing information (if available)
        - Target market
        - Unique selling points
        - Company positioning
        
        Respond in JSON format:
        {
            "company_name": "...",
            "products": [
                {"name": "...", "description": "...", "features": [...]}
            ],
            "pricing_info": "...",
            "target_market": "...",
            "key_features": [...],
            "positioning": "..."
        }
        """
    
    def get_links_user_prompt(self, url: str) -> str:
        """Build prompt for link selection (Day 5 pattern)"""
        user_prompt = f"""
        Here are links from competitor website {url}
        Select links relevant for competitive analysis (products, pricing, features).
        Respond with full https URLs in JSON format.
        
        Links:
        """
        links = fetch_website_links(url)
        user_prompt += "\n".join(links)
        return user_prompt
    
    def select_relevant_links(self, url: str) -> dict:
        """Select relevant links for analysis (Day 5 pattern)"""
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.link_system_prompt},
                {"role": "user", "content": self.get_links_user_prompt(url)}
            ],
            response_format={"type": "json_object"}
        )
        result = json.loads(response.choices[0].message.content)
        return result
    
    def extract_competitor_data(self, url: str) -> dict:
        """Extract structured competitor data (Day 5 pattern)"""
        # Step 1: Get landing page content
        landing_content = fetch_website_contents(url)
        
        # Step 2: Select relevant links
        relevant_links = self.select_relevant_links(url)
        
        # Step 3: Aggregate content (Day 5 pattern)
        aggregated_content = f"## Landing Page:\n\n{landing_content}\n## Key Pages:\n"
        for link in relevant_links.get('links', []):
            try:
                page_content = fetch_website_contents(link["url"])
                aggregated_content += f"\n\n### {link['type']}:\n{page_content}"
            except Exception as e:
                aggregated_content += f"\n\n### {link['type']}:\n(Error fetching: {str(e)})"
        
        # Step 4: Extract structured data (Day 5 pattern)
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.extraction_system_prompt},
                {"role": "user", "content": aggregated_content}
            ],
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        result['url'] = url
        return result
    
    def compare_competitors(self, competitor_urls: list, your_company_url: str = None) -> dict:
        """Compare multiple competitors (Day 5 pattern)"""
        print(f"Analyzing {len(competitor_urls)} competitors...")
        
        # Extract data from each competitor
        competitor_data = []
        for url in competitor_urls:
            print(f"Processing {url}...")
            try:
                data = self.extract_competitor_data(url)
                competitor_data.append(data)
            except Exception as e:
                print(f"Error analyzing {url}: {e}")
                continue
        
        # Get your company data if provided
        your_company_data = None
        if your_company_url:
            print(f"Processing your company: {your_company_url}...")
            try:
                your_company_data = self.extract_competitor_data(your_company_url)
            except Exception as e:
                print(f"Error analyzing your company: {e}")
        
        # Generate comparison report
        comparison_system_prompt = """
        You are a competitive intelligence analyst.
        Compare multiple competitors and provide insights.
        Create a comprehensive competitive analysis report in markdown format.
        Include:
        - Feature comparison matrix
        - Pricing comparison
        - Market positioning analysis
        - Strengths and weaknesses
        - Competitive gaps and opportunities
        """
        
        comparison_data = {
            "competitors": competitor_data
        }
        if your_company_data:
            comparison_data["your_company"] = your_company_data
        
        user_prompt = f"""
        Analyze and compare these competitors:
        
        {json.dumps(comparison_data, indent=2)}
        
        Generate a comprehensive competitive intelligence report.
        """
        
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": comparison_system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        
        return {
            "report": response.choices[0].message.content,
            "competitor_data": competitor_data,
            "your_company_data": your_company_data
        }
    
    def generate_report(self, competitor_urls: list, your_company_url: str = None):
        """Generate and display competitive intelligence report"""
        comparison = self.compare_competitors(competitor_urls, your_company_url)
        display(Markdown(comparison["report"]))
        return comparison

## Example 1: Analyze Single Competitor

In [5]:
# Initialize analyzer
analyzer = CompetitiveIntelligence()

# Analyze single competitor
competitor_data = analyzer.extract_competitor_data("https://huggingface.co")

# Display results
print(f"Company: {competitor_data.get('company_name')}")
print(f"Products: {len(competitor_data.get('products', []))}")
print(f"Key Features: {competitor_data.get('key_features', [])[:3]}")

Company: Hugging Face
Products: 6
Key Features: ['Open-source ML hub to host, discover, and experiment with models, datasets, and Spaces', '2M+ models and datasets catalog', 'Multi-modal support (text, image, video, audio, 3D)']


## Example 2: Compare Multiple Competitors

In [6]:
# Compare competitors
competitors = [
    "https://huggingface.co",
    "https://anthropic.com"
]

# Generate comparison report
result = analyzer.generate_report(competitors)

Analyzing 2 competitors...
Processing https://huggingface.co...
Processing https://anthropic.com...


Competitive Intelligence Report
Hugging Face vs Anthropic
Scope: Comparative analysis of the given Hugging Face suite and Anthropic offerings, focusing on product features, pricing, market positioning, strengths/weaknesses, and strategic gaps/opportunities.

Date: 2026-02-17

1) Executive summary
- Hugging Face excels as an open, collaboration-first platform with strong emphasis on open-source ethos, model/dataset hosting, and centralized, managed deployment options (Inference Endpoints, Spaces). It also targets enterprises with governance and security features (Enterprise Hub) plus scalable compute and private data capabilities.
- Anthropic positions Claude and its extended ecosystem around safety, reliability, and enterprise-grade workflows. It emphasizes agent-enabled coding and knowledge-work, deep integrations with common tools, and a Developer Platform to build on top of Claude across cloud environments. Lulu: pricing is token-based for core models, plus subscription tiers (Max, Team, Enterprise) for scale and governance.
- Competitive takeaway: Hugging Face dominates collaboration, model/dataset management, and multi-modality in an open ecosystem with strong enterprise governance. Anthropic dominates safety, reliability, developer tooling, and enterprise-ready deployments with strong governance and cross-cloud reach. The main gaps/choices revolve around governance depth, data residency, on-prem capabilities, cost models, and the breadth of agent-centric workflows.

2) Market positioning at a glance
- Hugging Face
  - Positioning: The leading open, community-driven AI platform that merges collaborative ML with enterprise-grade security and managed compute. Focused on rapid collaboration, deployment, and scaling of models, datasets, and interactive apps (Spaces).
  - Target markets: Individual developers/researchers, small-to-mid teams, and large enterprises seeking open-source collaboration with enterprise-grade deployment, security, and governance.
- Anthropic
  - Positioning: Safety-first, reliability-first AI designed for enterprise-grade workflows, coding, and agent-driven tasks. Emphasizes governance, privacy, and safe/interpretability-focused AI with a Developer Platform to build and scale across organizations.
  - Target markets: Business teams, developers, and large enterprises across customer support, education, finance, government, healthcare, life sciences, nonprofits; organizations requiring safety, auditability, and scalable AI workflows.

3) Feature comparison matrix (selected capabilities)
Note: This matrix evaluates core capabilities across Hugging Face’s four primary products and Anthropic’s main offerings. “✓” indicates strong coverage, “Partial” indicates partial/limited coverage, “No” indicates not primarily supported, and “N/A” indicates not applicable within the product’s scope.

- Key:
  - Hub = Hugging Face Hub
  - Endpoints = Inference Endpoints
  - Spaces = Spaces
  - Enterprise Hub = Enterprise Hub (Team/Enterprise)
  - Claude family = Anthropic Claude and related products
  - Platform = Claude Developer Platform

Capability area | Hub | Endpoints | Spaces | Enterprise Hub | Claude | Claude Code | Cowork | Claude Developer Platform | Claude Opus 4.6 | Claude Sonnet 4.5 | Claude Haiku 4.5 | Max plan | Team plan | Enterprise plan
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---
Open-source model/dataset hosting | ✓ | N/A | N/A | ✓ (private datasets, governance) | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A
Git-based collaboration / versioning | ✓ | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A
Public & private models/datasets | ✓ (public/private) | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A
Datasets viewer / data tooling | ✓ (datasets viewer) | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A
Multi-modality support | ✓ (text, image, video, audio, 3D) | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A | N/A
Managed deployment / one-click deployment | N/A | ✓ | N/A | ✓ | N/A | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
Autoscaling / pay-per-use compute | ✓ (via compute options) | ✓ (per-minute pricing) | N/A | ✓ | N/A | N/A | N/A | ✓ (API-based) | N/A | N/A | N/A | N/A | N/A | N/A
Observability / logs & metrics | ✓ | ✓ | N/A | ✓ | N/A | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
Secure access to model weights / private access | ✓ | N/A | N/A | ✓ | N/A | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
Single Sign-On (SSO) / SAML | ✓ | N/A | N/A | ✓ | N/A | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
Audit logs / governance / policy mgmt | ✓ | N/A | N/A | ✓ | N/A | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
Private storage / regional data controls | ✓ | N/A | N/A | ✓ (region selection) | N/A | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
Advanced compute options / GPUs | ✓ | N/A | N/A | ✓ | N/A | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
APIs for model deployment | ✓ | ✓ | N/A | ✓ | ✓ (API) | N/A | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A
Developer tooling / platform access | ✓ (ecosystem docs) | N/A | N/A | ✓ | ✓ (Claude Developer Platform) | ✓(for coding) | N/A | ✓ | N/A | N/A | N/A | N/A | N/A | N/A

Notes:
- Hugging Face products are organized as a portfolio: Hub (hosting/collaboration), Inference Endpoints (production deployment), Spaces (interactive apps), and Enterprise Hub (security/governance/SSO/audit). The table shows coverage where clearly stated in the provided data. Spaces emphasizes zero-GPU hosting and multi-modal app hosting.
- Anthropic products span Claude across general-use, coding, desktop integration, and developer tooling (Claude Developer Platform), plus long-context capabilities in Opus, Sonnet, Haiku variants, and enterprise/team plans with governance metrics.

2b) Pricing comparison (high level)
- Hugging Face
  - Hub/Enterprise: Pricing tiers include PRO (~$9 per user/month), Team (~$20 per user/month), and Enterprise (~$50+ per user/month) with custom onboarding and contract options.
  - Inference Endpoints: Self-serve pay-as-you-go pricing (per-minute usage). Rates vary by engine and configuration.
  - Notes: The pricing model emphasizes per-user subscriptions for collaboration/enterprise features and usage-based pricing for production deployments.
- Anthropic
  - Claude Opus 4.6: Starts at $5 per 1,000,000 input tokens and $25 per 1,000,000 output tokens; savings up to 90% via prompt caching; up to 50% via batch processing.
  - Claude Sonnet 4.5: Starts at $3 per 1,000,000 input tokens and $15 per 1,000,000 output tokens; similar savings mechanisms.
  - Claude Haiku 4.5: Starts at $1 per 1,000,000 input tokens and $5 per 1,000,000 output tokens; lower-cost option optimized for latency-sensitive use.
  - Max plan: Bundles Claude desktop/mobile apps with Claude Code and promises up to 20x more usage per session than Pro.
  - Team plan: For collaboration across teams; productivity gains cited.
  - Enterprise plan: Secure, enterprise-grade deployments with governance and scalable deployment; enterprise metrics (broad adoption, agents deployed, etc.) cited.
  - Notes: Token-based pricing with volume discounts via prompt caching and batch processing; multiple plan tiers catering to individuals, teams, and enterprises; cross-cloud availability implied.

3) Market positioning: deeper analysis
- Hugging Face
  - Core value proposition: Open-source, collaborative ML with centralized hosting (models, datasets, Spaces) and managed production options. Emphasis on community, transparency, and rapid experimentation with governance for teams/enterprises.
  - Competitive edge: Largest emphasis on multi-modal, open weights/datasets, and a broad ecosystem enabling rapid iteration and sharing. Enterprise Hub adds governance, SSO, private data, and audit capabilities to meet enterprise needs.
  - Risks/constraints: If enterprise buyers prioritize strict data residency, on-prem deployments, or highly specialized safety controls beyond governance tooling, Hugging Face may need deeper capabilities or partnerships to meet those requirements.
- Anthropic
  - Core value proposition: Safety-first, reliable AI with robust developer tooling and enterprise-grade deployment. Emphasizes agentivity, long-context capabilities, and deep integrations with common productivity tools (Chrome, Slack, Excel, PowerPoint).
  - Competitive edge: Strong focus on safety, interpretability, and governance; a breadth of Claude variants tailored for coding, agents, and enterprise workflows; cross-cloud availability via the Claude Developer Platform.
  - Risks/constraints: Token-based pricing can be a barrier at scale if usage is heavy; dependency on token economics may affect cost predictability; market perception of safety features vs. speed/cost tradeoffs for some use cases.

4) Strengths and weaknesses (high-signal)
- Hugging Face
  - Strengths
    - Open-source, community-driven; vast ecosystem of models, datasets, and Spaces.
    - Strong collaboration tooling with Git-based workflows and private data governance options.
    - Flexible deployment options (Hub, Inference Endpoints, Spaces) with multi-modality support.
    - Enterprise governance features (SSO, SAML, audit logs, policy management) and private data handling.
  - Weaknesses
    - Production-scale safety/compliance features may be less prescriptive than a safety-first vendor’s core offering.
    - Token-based pricing for production endpoints can be cost-uncertain without clear usage patterns.
    - On-prem or fully isolated data residency options are not explicitly detailed in the provided data (though region storage is mentioned in Enterprise Hub).

- Anthropic
  - Strengths
    - Safety-first, reliability, interpretability, and steerability baked into design.
    - Broad developer tooling (Claude Developer Platform) and multi-modal capabilities across coding, agents, and enterprise workflows.
    - Strong enterprise sentiment with governance metrics and cross-cloud availability (Bedrock/Vertex/Foundry compatibility).
    - Long-context capabilities (Opus 4.6) enabling sophisticated agent reasoning and coding workflows.
  - Weaknesses
    - Token-based pricing can be higher at scale for token-heavy use cases; predictability depends on usage patterns.
    - Open collaboration and community ecosystem weaker than Hugging Face in terms of open-source model/dataset sharing (perceived tradeoff for safety).
    - Desktop integration (Cowork) is part of the Claude ecosystem but may introduce separate cost/management considerations for large teams.

5) Competitive gaps and opportunities (what each could exploit or address)
- Gaps in Hugging Face relative to Anthropic
  - Safety and governance depth: While Enterprise Hub provides governance features, Anthropic’s safety-focused framing and long-context agent capabilities may outpace Hugging Face in highly regulated domains (e.g., healthcare, finance) where formal risk frameworks are required.
  - On-prem / fully isolated deployments: While Enterprise Hub provides private storage, explicit on-prem or fully air-gapped deployments are not described. Opportunity to expand on-prem capabilities or more granular data residency controls.
  - Deep safety tooling: Differentiating with built-in safety/steering tooling for end-users and governance policies could be expanded beyond audit logs and SSO.
- Gaps in Anthropic relative to Hugging Face
  - Open-source collaboration and community ecosystem: Hugging Face’s open weights, datasets, and community-driven Spaces are a powerful magnet for developers. Anthropic could explore more open collaboration or community tooling around safer-by-design models without compromising safety.
  - Modularity and multi-modality ecosystem: Hugging Face’s breadth across models, datasets, and Spaces across modalities could serve as a broader platform if Anthropic expands model hosting or open adapters to integrate with more third-party tools.
  - Pricing transparency and predictability: Token-based pricing is strong for certain usage profiles but can be opaque for larger teams. A clearer, tiered cost model with usage caps or bundled governance features could help large-scale adoption.

6) Strategic recommendations (actionable guidance)
- For Hugging Face
  - Deepen enterprise governance and data residency options: Expand explicit on-premizable components or tighter data residency controls; improve auditability with configurable governance workflows for sensitive industries.
  - Expand safety/compliance tooling: Build modular safety controls, model cards, and steerability policies integrated into the Enterprise Hub to complement existing SSO/audit features.
  - Accelerate conversion of open-source momentum into enterprise scale: Offer clearer cost predictability for large-scale production use (e.g., tiered per-user plus predictable per-minute for endpoints, with caps and enterprise SLAs).
  - Grow the Spaces ecosystem for enterprise-grade apps: Invest in approved templates for regulated domains (e.g., healthcare, finance) and provide security reviews for community apps.
- For Anthropic
  - Extend open collaboration to broaden ecosystem: Consider offering safer-by-design open templates, examples, or “AI safety kits” to enable developers to build responsibly while leveraging Claude in common workflows.
  - Improve cost predictability and value realization: Provide more transparent enterprise pricing tiers, volume-based discounts, and better alignment between usage patterns (coding, agents, long-running tasks) and cost.
  - Expand multi-cloud integration and on-prem options: While cross-cloud availability exists, explicit on-prem/offline deployment options could win contracts in regulated sectors requiring data sovereignty.
  - Strengthen developer experience: Continue to simplify the Claude Developer Platform with richer tooling, code samples, and rapid-start guides for common enterprise workflows (customer support, education, healthcare).

7) Quick synthesis by product category (practical takeaways)
- Open-source collaboration and hosting
  - Hugging Face is strongest in open collaboration, hosting of weights/datasets, and community-driven innovation.
  - Anthropic prioritizes safety over open collaboration; limited emphasis on community hosting of models/datasets.
- Production deployment and governance
  - Hugging Face provides robust enterprise governance features (SSO, audit logs, private storage) and managed deployment via Endpoints.
  - Anthropic emphasizes safety, long-context reasoning for agents, and enterprise-scale governance through its Enterprise plan.
- Developer tooling and AI agents
  - Hugging Face offers a broad ecosystem for models/datasets/spaces; Claude offers agentic capabilities but is more platform-centric around safety and enterprise workflows.
  - Anthropic shines with Claude Developer Platform, Opus/Sonnet/Haiku variants, and desktop integration via Cowork for local file/tool access.
- Pricing dynamics
  - Hugging Face: Mixed model with per-user Enterprise pricing and pay-as-you-go Endpoints pricing.
  - Anthropic: Token-based pricing across Opus/Sonnet/Haiku with discounts via caching/batching; subscription bundles (Max) and team/enterprise plans.

8) Conclusion
- Hugging Face and Anthropic serve overlapping but distinct appeals. Hugging Face is best for organizations seeking an open, collaborative, multi-modality ML platform with strong governance and scalable production deployment options. Anthropic is best for organizations prioritizing safety, interpretability, and enterprise-grade workflows with sophisticated agent tooling and cross-cloud compatibility.
- For buyers, the choice depends on priority: open collaboration and ecosystem breadth (Hugging Face) vs safety-centric enterprise deployment with developer tooling and long-context agent capabilities (Anthropic). Consider hybrid strategies as appropriate (e.g., leveraging Hugging Face for model/dataset experimentation and Anthropic for safety-critical production workflows or agent-enabled tasks).

9) Appendix / sources
- Provided competitor data (JSON) detailing Hugging Face products (Hub, Inference Endpoints, Spaces, Enterprise Hub), pricing, target markets, and features.
- Provided Anthropic data detailing Claude family (Claude, Claude Code, Cowork, Claude Developer Platform, Claude Opus/Sonnet/Haiku), pricing, target markets, and enterprise features.

If you’d like, I can convert this into a slide-ready deck, or tailor the matrices to your preferred level of granularity (e.g., adding a SWOT per product or a one-page executive brief).

## Example 3: Access Structured Data

In [7]:
# Access structured competitor data
for comp in result["competitor_data"]:
    print(f"\n{comp.get('company_name', 'Unknown')}:")
    print(f"  URL: {comp.get('url')}")
    print(f"  Products: {len(comp.get('products', []))}")
    print(f"  Key Features: {len(comp.get('key_features', []))}")
    if comp.get('pricing_info'):
        print(f"  Pricing: {comp.get('pricing_info')[:100]}...")


Hugging Face:
  URL: https://huggingface.co
  Products: 4
  Key Features: 9
  Pricing: PRO: $9 per user/month; Team: $20 per user/month; Enterprise: starting at $50 per user/month with cu...

Anthropic:
  URL: https://anthropic.com
  Products: 10
  Key Features: 10
  Pricing: Opus 4.6 pricing: starting at $5 per million input tokens and $25 per million output tokens, with up...
